In [1]:
import numpy as np
import random
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [3]:
# Parameter
alpha = 0.001  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_decay = 0.995  # Decay rate for exploration
min_epsilon = 0.01  # Minimum exploration rate
batch_size = 64  # Batch size for training
num_episodes = 1000  # Number of training episodes


In [4]:

# Environment setup
grid_size = 5  # 5x5 grid
goal_state = (4, 4)  # Goal position
state_size = grid_size * grid_size
action_size = 4  # Four possible actions: up, down, left, right

# Replay memory
memory = deque(maxlen=2000)


In [6]:
# Neural network model
def build_model():
    model = Sequential()
    model.add(Dense(24, input_dim=state_size, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=alpha))
    return model

model = build_model()
# target_model = build_model()

ValueError: Argument(s) not recognized: {'lr': 0.001}

In [2]:




# Functions to convert state to one-hot encoding
def state_to_one_hot(state):
    one_hot = np.zeros((grid_size, grid_size))
    one_hot[state] = 1
    return one_hot.flatten()

# Reward function
def get_reward(state):
    if state == goal_state:
        return 100
    return -1

# State transition function
def get_next_state(state, action):
    x, y = state
    if action == 0 and x > 0:  # Up
        x -= 1
    elif action == 1 and x < grid_size - 1:  # Down
        x += 1
    elif action == 2 and y > 0:  # Left
        y -= 1
    elif action == 3 and y < grid_size - 1:  # Right
        y += 1
    return (x, y)

# Training function
def train_model():
    if len(memory) < batch_size:
        return
    minibatch = random.sample(memory, batch_size)
    for state, action, reward, next_state, done in minibatch:
        target = reward
        if not done:
            target = reward + gamma * np.amax(target_model.predict(state_to_one_hot(next_state).reshape(1, -1))[0])
        target_f = model.predict(state_to_one_hot(state).reshape(1, -1))
        target_f[0][action] = target
        model.fit(state_to_one_hot(state).reshape(1, -1), target_f, epochs=1, verbose=0)

# Training loop
for episode in range(num_episodes):
    state = (0, 0)  # Start position
    total_reward = 0
    for time in range(200):
        if random.uniform(0, 1) < epsilon:
            action = random.choice(range(action_size))
        else:
            action = np.argmax(model.predict(state_to_one_hot(state).reshape(1, -1))[0])

        next_state = get_next_state(state, action)
        reward = get_reward(next_state)
        done = next_state == goal_state
        total_reward += reward
        memory.append((state, action, reward, next_state, done))
        state = next_state

        train_model()

        if done:
            break

    # Decay epsilon
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    # Update target network
    if episode % 10 == 0:
        target_model.set_weights(model.get_weights())

    print(f"Episode: {episode + 1}/{num_episodes}, Total Reward: {total_reward}, Epsilon: {epsilon:.2f}")

print("Training finished!")


d:\cursor_code\python\Lstm\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Argument(s) not recognized: {'lr': 0.001}